In [1]:
#this implementation has been derived from http://bit.ly/QLSCommunity

In [ ]:
#from qiskit import IBMQ

In [ ]:
#IBMQ.enable_account(token=...)

In [ ]:
%matplotlib inline

import time
import numpy as np
import progressbar_renamed as progressbar
import copy
import networkx as nx
from networkx.algorithms import bipartite
from operator import itemgetter
from dimod import BinaryQuadraticModel
from dimod import Vartype

from dimod.reference.samplers import SimulatedAnnealingSampler
import neal

import import_ipynb
import QAOA_for_QLS as qaoa
import Side_Calc_GB1 as sc

In [4]:
def top_gains_populate_subset(lis, subset_size, gains): #function for population subset, alternative approaches would be possible here
    if len(lis) <= subset_size:
        return sorted(lis)
    return [
        x[0] for x in sorted(gains.items(), key=itemgetter(1))[-subset_size:]
    ]

In [16]:
def iteration_step(M, curr_solution, subset, method='sim_anneal', backend='simulator'): #local optimization for subset and fixed boardering vertices
    lis=range(len(M))
    n=len(subset)
    subset2global = dict((x, subset[x]) for x in range(n))
    global2subset = dict((subset[x], x) for x in range(n))
    
    C = [0.0] * n #create linear term representing fixed vertices
    for i in subset:
        for j in set(lis) - set(subset):
            C[global2subset[i]] += 2 * M[i,j] * curr_solution[j]
                                                           
    indices = []
    for i in subset:
        indices.append(i)
    indices=np.array(indices) #indices to keep for subset optimization
    
    if method=='qaoa':
        result = qaoa.qaoa_basic(n, M[np.ix_(indices, indices)], C, backend)
        optimized_subset=result[0].x
        ex_time=result[1]
    
    if method=='sim_anneal':
        st=time.time()
        for i in range(n):
            C[i]=(-1)*C[i]
        quadr={}
        s = ['']*n
        for i in range(n):
            s[i]="s_"+str(i)
        linear=dict((s[i],C[i]) for i in range(n))
            
        for i in range(n):
            for j in range(n):
                    quadr[(s[i],s[j])]=(-1)*M[subset2global[i],subset2global[j]]
                    
        bqm = BinaryQuadraticModel(linear, quadr, 0, Vartype.BINARY)
        sampler = neal.SimulatedAnnealingSampler()
        response = sampler.sample(bqm)
            
        ss=0
        i=0
        en=0
        conf_classic=[]

        for datum in response.data(['sample', 'energy']):   
            if en>datum.energy:
                    en=datum.energy
                    ss=i
            con=[]
            for h in range(n):
                con.append(datum.sample[s[h]])

            conf_classic.append(con)
                    
            result=np.array(conf_classic[ss])
            if(i >= 10):
                break
            i += 1
        et=time.time()
        ex_time=(et-st)*1000 #time in milliseconds 
        optimized_subset=result
        
    if 0 in optimized_subset:
        optimized_subset = [-1 if x == 0 else 1 if x == 1 else 'Error' for x in optimized_subset]
        
    for i in range(0, len(optimized_subset)):
        curr_solution[subset2global[i]] = optimized_subset[i]
    return curr_solution, ex_time   
    

   

def QLS(M, method='sim_anneal', backend='simulator', size_of_iteration=12,stopping_criteria=3):
    
    run_time=0
    n=len(M)
    lis=list(range(n))

    # random initial guess
    curr_solution = [1 - 2 * x for x in list(np.random.randint(2, size=n))]
    curr_solution=np.array(curr_solution)
    if 0 in curr_solution:
        curr_solution = [-1 if x == 0 else 1 if x == 1 else 'Error' for x in curr_solution]
    
    
    curr_energy = sc.compute_energy(M, curr_solution)
    
    all_time_best_solution = curr_solution
    all_time_best_energy = curr_energy

    visited = set()
    it = 0
    it_stuck = 0
    all_energies = []
    
    while (n - len(visited)):
        it += 1
        #print("Iteration: ", it)
        if it_stuck > stopping_criteria: 
            break
        gains_list=[]
        for v in lis:
            gains_list.append(sc.compute_gain(M, curr_solution, v, True))
        #print(gains_list)
        gains = {v: gain for gain, v in gains_list} 
       #print(gains)
        
        
        subset = top_gains_populate_subset(lis, size_of_iteration, gains) #identify most promising subset based on gains
        
        iter_step = iteration_step(M, copy.deepcopy(curr_solution), list(subset), method, backend) #optimize for subset and fixed outer-subset vertices
        cand_solution=iter_step[0]
        run_time+=iter_step[1]
        #print(iter_step[1])
        cand_energy = sc.compute_energy(M, cand_solution)
        
        #print('it', it, 'cand_energy', cand_energy, 'curr_best',
              #all_time_best_energy)
        #return cand_energy, cand_solution
        
        all_energies.append({
            'it': it,
            'cand_energy': cand_energy,
            'curr_best': all_time_best_energy
        })
        if cand_energy > curr_energy: #update if (modified) modularity has improved
            curr_solution = cand_solution
            curr_energy = cand_energy
            it_stuck = 0
        else:
            it_stuck += 1
        if curr_energy > all_time_best_energy:
            all_time_best_solution = curr_solution
            all_time_best_energy = curr_energy
        
        #this version does not include an additional break-condition in case a very good solution has been found but rather keeps improving until it_stuck>3
    return (all_time_best_solution, run_time, all_time_best_energy,  it,
            all_energies)
        